In [2]:
import pickle
import random
import collections
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [18]:
def loadfile(path):
    with open(path,'rb') as picklefile:
        file = pickle.load(picklefile, encoding='latin1')
    return file

In [3]:
with open('./data/2016_q2_newman-weights.pickle','rb') as picklefile:
    q2_newman_w = pickle.load(picklefile, encoding='latin1')

In [4]:
with open('./data/q1_4000_users.pickle', 'rb') as picklefile:
    users = pickle.load(picklefile, encoding='latin1')

In [5]:
with open('./data/newman_cn.pickle', 'rb') as picklefile:
    q1_cn = pickle.load(picklefile, encoding='latin1')

In [6]:
with open('./data/newman_adam.pickle', 'rb') as picklefile:
    q1_aa = pickle.load(picklefile, encoding='latin1')

In [7]:
with open('./data/newman_jaccard.pickle', 'rb') as picklefile:
    q1_jc = pickle.load(picklefile, encoding='latin1')

In [8]:
with open('./data/newman_pa.pickle', 'rb') as picklefile:
    q1_pa = pickle.load(picklefile, encoding='latin1')

In [9]:
with open('./data/2016_q1_newman-weights.pickle', 'rb') as picklefile:
    q1_newman_w = pickle.load(picklefile, encoding='latin1')

In [10]:
def check_accuracy(q1_w, q2_w, score, core_pairs):
    new_edges = set(q2_w.keys()).difference(q1_w.keys()).intersection(set(core_pairs))
    n = int(len(new_edges))
    predicted_new_edges = [(a,b) for a,b in list(zip(score.keys(), score.values())) if a not in q1_w.keys()]
    sorted_pred = sorted(predicted_new_edges, key=lambda x:x[1], reverse=True)[:n]
    predicted_edges = set([a for a,b in sorted_pred])
    accurate_prediction = set(predicted_edges).intersection(new_edges)
    return len(accurate_prediction)/len(predicted_edges)

In [11]:
check_accuracy(q1_newman_w, q2_newman_w, q1_jc, q1_pa.keys())

0.15958474167069048

In [166]:
check_accuracy(q1_newman_w, q2_newman_w, q1_pa, q1_pa.keys())

0.019099737110359997

In [167]:
check_accuracy(q1_newman_w, q2_newman_w, q1_cn, q1_pa.keys())

0.15694243253393422

In [168]:
check_accuracy(q1_newman_w, q2_newman_w, q1_aa, q1_pa.keys())

0.16659960298299265

## Construct dataframe for supervised learning

In [13]:
# Get new core edges
nce = list(set(q1_pa.keys()).difference(q1_newman_w.keys()))

In [14]:
def combine_scores(edges, score_list):
    scores = np.zeros((len(edges), len(score_list)))
    for i in range(len(edges)):
        for j in range(len(score_list)):
            scores[i][j] = score_list[j][edges[i]]
    return scores

In [16]:
X = combine_scores(nce, [q1_aa, q1_cn, q1_jc, q1_pa])
y = [a in q2_newman_w.keys() for a in nce]

## Combine with user behavior and demographic features

In [53]:
user_details = loadfile('./data/user_details.pickle')

In [48]:
user_country = user_details.country_code.to_dict()
user_created = user_details.months_since_1970.to_dict()
user_commit = user_details.commit_times.to_dict()

In [54]:
def get_user_details(edges, detail_list):
    details = np.zeros((len(edges), len(detail_list)))
    #print (f'{len(edges)/1000}k node pairs to process')
    for i in range(len(edges)):
        #if not i%10000:
        #    print (f'Processing {i/1000}k th node pair')
        uid = edges[i][0]
        vid = edges[i][1]
        details[i][0] = int(detail_list[0][uid] == detail_list[0][vid])
        details[i][1] = abs(detail_list[1][uid] - detail_list[1][vid])
        details[i][2] = min(detail_list[2][uid], detail_list[2][vid]) == 0
    return details

In [ ]:
details = get_user_details(nce, [user_country, user_created, user_commit])
X = np.concatenate((X,details), axis=1)

## Build a baseline model with logistic regression

In [116]:
# Divide data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234, stratify=y)
# Downsample the training set
rus = RandomUnderSampler()
X_under, y_under = rus.fit_sample(X_train, y_train)
ssX = StandardScaler()
X_scaled = ssX.fit_transform(X_under)

In [63]:
models = [
          ('logistic', LogisticRegression),
          ('tree', DecisionTreeClassifier),
          ('forest', RandomForestClassifier),
          ('xgboost', XGBClassifier)
         ]

param_choices = [    
    {
        'C': np.logspace(-3, 6, 12),
        'penalty': ['l1', 'l2']
    },
    {
        'max_depth': [1,2,3,4,5],
        'min_samples_leaf': [1,3,5]
    },
    {
        'criterion':['gini','entropy'],
        'n_estimators': [40,50,60],
        'min_samples_leaf':[1,3],
        'min_samples_split':[2,5]
    },
    {
        'max_depth': [3,4,5],
        'n_estimators': [1, 50, 100,200],
        'objective':['binary:logistic']
    }
]

grids = {}
for model_info, params in zip(models, param_choices):
    name, model = model_info
    grid = GridSearchCV(model(), params, scoring='accuracy', cv=5, n_jobs=-1)
    grid.fit(X_scaled, y_under)
    s = "{}: best score: {}".format(name, grid.best_score_)
    print(s)
    grids[name] = grid

logistic: best score: 0.8384238057828278
tree: best score: 0.8720420011879898
forest: best score: 0.8822932035486405


/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py

/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py

/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py

/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py

/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/songlin/anaconda2/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py

xgboost: best score: 0.8779819502193948


In [89]:
X_test_scaled = ssX.transform(X_train)

In [90]:
y_test_pred = [x[1] for x in grids['tree'].predict_proba(X_test_scaled)]

### Check training accuracy with the same method with similarity score

In [64]:
def check_prediction_accuracy(pred_prob, y):
    n = sum(y)
    sorted_index = [a for a,b in sorted(enumerate(pred_prob), key=lambda x:x[1], reverse=True)][:n]
    true_index = [a for a,b in enumerate(y) if b == 1]
    matches = set(sorted_index).intersection(true_index)
    return len(matches)/n

In [126]:
X_test_scaled = ssX.transform(X_train)
y_test_pred = [x[1] for x in grids['tree'].predict_proba(X_test_scaled)]
for model in grids.keys():
    X_train_scaled = ssX.transform(X_train)
    y_train_pred = [x[1] for x in grids[model].predict_proba(X_train_scaled)]
    print (f'Model {model} has an accuracy of {check_prediction_accuracy(y_train_pred, y_train)}')

Model logistic has an accuracy of 0.18523060415029988
Model tree has an accuracy of 0.1761099082182069
Model forest has an accuracy of 0.25813868822932035
Model xgboost has an accuracy of 0.2135507482419667


## Apply the best model on test data

In [123]:
X_test_scaled = ssX.transform(X_test)
y_test_pred = [x[1] for x in grids['forest'].predict_proba(X_test_scaled)]

In [124]:
check_prediction_accuracy(y_test_pred, y_test)

0.1964948361425314